In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
import re
import string
import nltk
from nltk.corpus import stopwords

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [5]:
df_train.shape

(7613, 5)

In [6]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
imputer = SimpleImputer(strategy='most_frequent')
df_train[['keyword']] = imputer.fit_transform(df_train[['keyword']])


In [8]:
df_train.head()

,id,keyword,location,text,target
0,1,fatalities,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,fatalities,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,fatalities,NaN,All residents asked to 'shelter in place' are ...,1
3,6,fatalities,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,fatalities,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
len(df_train['location'].unique())

3342

In [10]:
imputer2 = SimpleImputer(strategy = 'most_frequent')
df_train['location'] = imputer2.fit_transform(df_train[['location']])[:, 0]

In [11]:
def text_preprocess(text):
    text = text.lower()
    text = text.strip()
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [12]:
text = df_train['text'].to_list()
labels = df_train['target'].to_list()

In [13]:
len(text)/len(labels) #same lengths 

1.0

In [14]:
text = [text_preprocess(t) for t in text] #apply preprocess function
text = [stopword(t) for t in text] #remove stop words

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = TfidfVectorizer(max_features = 10000, ngram_range = (1,2))
x_train = vectorizer.fit_transform(text)

model = LogisticRegression(max_iter = 1000)
model.fit(x_train, labels)

LogisticRegression(max_iter=1000)

In [16]:
test_texts = df_test['text']
test_texts = [text_preprocess(t) for t in test_texts]
test_texts = [stopword(t) for t in test_texts]

x_test = vectorizer.fit_transform(test_texts)

In [17]:
preds = model.predict(x_test)

In [18]:
submission = pd.DataFrame({
    'id' : df_test['id'].to_list(),
    'target' : preds
})

submission.to_csv('submission.csv', index = False)